In [1]:

pip install joblib


Note: you may need to restart the kernel to use updated packages.


# 01 Preprocessing

In [1]:
## Import libraries
import os
import scanpy as sc
import anndata as ad
import numpy as np
import pandas as pd
from scipy import sparse
from scipy.io import mmread



In [2]:
import os
import scanpy as sc
import anndata as ad
import numpy as np
import pandas as pd
from scipy import sparse
from scipy.io import mmread
from joblib import Parallel, delayed

def process_chunk(chunk, barcodes, features):
    """
    Create an AnnData object for a given chunk of data.

    Parameters:
    - chunk: sparse matrix, the expression data for the chunk.
    - barcodes: array-like, the barcodes corresponding to the cells in the chunk.
    - features: array-like, the gene names.

    Returns:
    - AnnData object for the chunk.
    """
    adata = ad.AnnData(X=chunk, obs=pd.DataFrame(index=barcodes), var=pd.DataFrame(index=features))
    adata.var_names_make_unique()  # Ensure unique gene names
    return adata

def load_scrna_data(data_dir, chunk_size=5000):
    """
    Load single-cell RNA sequencing (scRNA-seq) data from specified directory.

    Parameters:
    - data_dir: str, path to the directory containing scRNA-seq data folders.
    - chunk_size: int, number of cells to process at a time (default is 5000).

    Returns:
    - AnnData object containing the combined scRNA-seq data.
    """
    adata_list = []  # List to store AnnData objects for each chunk
    for folder_name in os.listdir(data_dir):  # Iterate through each folder in the data directory
        folder_path = os.path.join(data_dir, folder_name)  # Construct full folder path
        if os.path.isdir(folder_path):  # Check if the path is a directory
            try:
                # Initialize paths for barcodes, features, and matrix files
                barcodes_path = None
                features_path = None
                matrix_path = None

                # Look for specific files in the folder
                for file in os.listdir(folder_path):
                    if 'barcodes' in file and file.endswith('.tsv'):
                        barcodes_path = os.path.join(folder_path, file)
                    elif 'features' in file and file.endswith('.tsv'):
                        features_path = os.path.join(folder_path, file)
                    elif 'matrix' in file and (file.endswith('.mtx') or file.endswith('.npz')):
                        matrix_path = os.path.join(folder_path, file)

                # Check if all required files are found
                if barcodes_path and features_path and matrix_path:
                    # Load barcodes and features
                    barcodes = pd.read_csv(barcodes_path, header=None, sep='	', dtype='str')[0].values
                    features = pd.read_csv(features_path, header=None, sep='	', dtype='str')[1].values

                    # Ensure gene names are unique
                    features = pd.Index(features).unique().tolist()

                    # Load the expression matrix
                    matrix = sparse.load_npz(matrix_path) if matrix_path.endswith('.npz') else mmread(matrix_path).tocsc()

                    # Report size before processing
                    print(f"Processing folder: {folder_name}")
                    print(f"Size of matrix before processing: {matrix.shape[0]} genes x {matrix.shape[1]} cells")

                    # Adjust features length to match the matrix rows
                    if len(features) != matrix.shape[0]:
                        print(f"Warning: Feature count ({len(features)}) doesn't match matrix rows ({matrix.shape[0]}). Matching the length.")
                        if len(features) > matrix.shape[0]:
                            features = features[:matrix.shape[0]]  # Truncate features if too long
                        else:  # Pad features if they're less than matrix rows
                            features = np.append(features, [f'Missing_Gene_{i}' for i in range(matrix.shape[0] - len(features))])

                    # Process matrix in smaller chunks to manage memory
                    n_cells = len(barcodes)
                    chunks = [matrix[:, i:i + chunk_size].T.astype(np.float32) for i in range(0, n_cells, chunk_size)]
                    barcodes_chunks = [barcodes[i:i + chunk_size] for i in range(0, n_cells, chunk_size)]

                    # Parallel processing of chunks
                    adata_list = Parallel(n_jobs=-1)(delayed(process_chunk)(chunk, barcodes_chunk, features) 
                                                       for chunk, barcodes_chunk in zip(chunks, barcodes_chunks))

                    # Report size after processing
                    combined_adata = ad.concat(adata_list, axis=0, join='outer')  # Concatenate all AnnData objects
                    print(f"Size of combined AnnData after processing: {combined_adata.n_vars} genes x {combined_adata.n_obs} cells")
                else:
                    print(f"Missing files in folder {folder_name}")  # Warn if files are missing
            except Exception as e:
                print(f"Error loading data from {folder_name}: {e}")  # Catch and print any errors

    # Return the combined AnnData object
    return ad.concat(adata_list, axis=0, join='outer')


# Load scRNA-seq data
scrna_data_path = '../data/raw/scRNA_raw/'  # Path to raw scRNA-seq data
scrna_adata = load_scrna_data(scrna_data_path)  # Call the function to load data
# Save the processed scRNA-seq data
scrna_adata.write('../data/processed/processed_scrna_adata.h5ad')  # Save the processed data

Processing folder: b89b306c-11a9-426a-bd42-5b4d0d122ce3
Size of matrix before processing: 33786 genes x 737280 cells
Size of combined AnnData after processing: 33786 genes x 737280 cells
Processing folder: bdd322a1-2acd-4ed6-a82a-31dd054f2b80
Size of matrix before processing: 33694 genes x 737280 cells
Size of combined AnnData after processing: 33694 genes x 737280 cells
Processing folder: d1553918-a4b7-40b2-b895-7e3c0af3033a
Size of matrix before processing: 33694 genes x 737280 cells
Size of combined AnnData after processing: 33694 genes x 737280 cells
Processing folder: d6ab59e1-7d70-4bac-ae4a-3f0ff3125114
Size of matrix before processing: 33694 genes x 737280 cells
Size of combined AnnData after processing: 33694 genes x 737280 cells
Processing folder: f1fb0fa0-fb26-4eb2-953a-f3699accee8c
Size of matrix before processing: 33781 genes x 737280 cells
Size of combined AnnData after processing: 33781 genes x 737280 cells
Processing folder: Patient_460_Tumor
Size of matrix before proces

In [3]:


# Load Visium data
visium_data_path = '../data/raw/Visium_raw/Human_Lung.h5'  # Path to raw Visium data
try:
    visium_adata = sc.read_10x_h5(visium_data_path)  # Load Visium data
    # Report size of Visium data
    print(f"Size of Visium data: {visium_adata.n_vars} genes x {visium_adata.n_obs} cells")
    
    # Ensure gene names are unique
    visium_adata.var_names_make_unique()  
    # Save the processed Visium data
    visium_adata.write('../data/processed/processed_visium_adata.h5ad')  # Save the processed Visium data
except Exception as e:
    print(f"Error loading Visium data: {e}")  # Catch and print any errors

print("Data processing complete.")

c:\Users\iqram\.conda\envs\integration\Lib\site-packages\anndata\_core\anndata.py:1758: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
c:\Users\iqram\.conda\envs\integration\Lib\site-packages\anndata\_core\anndata.py:1758: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


Size of Visium data: 18085 genes x 3858 cells
Data processing complete.
